In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
import imutils 

In [ ]:
# import video capture
cap = cv2.VideoCapture('/Users/andreibirladeanu/Documents/Data/play_videos/1073_play.mp4')

In [ ]:
cap.set(1,900)

In [ ]:
prototxtPath = "/Users/andreibirladeanu/Documents/Code/PhD_work/openpose_tf/deploy.prototxt.txt"
weightsPath = "/Users/andreibirladeanu/Documents/Code/PhD_work/openpose_tf/res10_300x300_ssd_iter_140000.caffemodel"

In [ ]:
detector = cv2.dnn.readNetFromCaffe(prototxtPath , weightsPath)

In [ ]:
ret, image = cap.read()
plt.imshow(image)

In [ ]:
base_img = image.copy()
original_size = base_img.shape
target_size = (300, 300)
image = cv2.resize(image, target_size)
aspect_ratio_x = (original_size[1] / target_size[1])
aspect_ratio_y = (original_size[0] / target_size[0])

In [ ]:
imageBlob = cv2.dnn.blobFromImage(image = image)

In [ ]:
detector.setInput(imageBlob)
detections = detector.forward()

In [ ]:
column_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]
detections_df = pd.DataFrame(detections[0][0], columns = column_labels)

In [ ]:
detections_df = detections_df[detections_df['is_face'] == 1]

In [ ]:
detections_df['left'] = (detections_df['left'] * 300).astype(int)
detections_df['bottom'] = (detections_df['bottom'] * 300).astype(int)
detections_df['right'] = (detections_df['right'] * 300).astype(int)
detections_df['top'] = (detections_df['top'] * 300).astype(int)

In [ ]:
detections_df

In [ ]:
detected_faces = []
for i, instance in detections_df.iterrows():
    confidence_score = str(round(100*instance["confidence"], 2))+" %"
    left = instance["left"]; right = instance["right"]
    bottom = instance["bottom"]; top = instance["top"]
    detected_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y) ,
    int(left*aspect_ratio_x):int(right*aspect_ratio_x)]
    detected_faces.append(detected_face)
    print("Id ",i,". Confidence: ", confidence_score)
plt.imshow(detected_face[:,:,::-1])
plt.show()

In [ ]:
plt.imshow(detected_faces[10])

In [ ]:
len(detected_faces[3])

In [ ]:
pip install deepface

In [ ]:
mp_drawing = mp.solutions.drawing_utils
# load face detection model
mp_face = mp.solutions.face_detection.FaceDetection(
    model_selection=1, # model selection
    min_detection_confidence=0.1 # confidence threshold
)

In [ ]:
image_input = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# get results
results = mp_face.process(image_input)

if not results.detections:
    print('No faces detected.')
else:
    for detection in results.detections: # iterate over each detection and draw on image
        mp_drawing.draw_detection(image, detection)


In [ ]:
from deepface import DeepFace
from retinaface import RetinaFace

In [ ]:
cv2.imwrite('img.jpg', image)

In [ ]:
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
face = DeepFace.detectFace(img_path = "img.jpg", target_size = (224, 224), detector_backend = backends[4])

In [ ]:
plt.imshow(face)

In [ ]:
face_detector = RetinaFace.build_model()

In [ ]:
def detect_face(face_detector, img, align = True):

    from retinaface import RetinaFace
    from retinaface.commons import postprocess

    #---------------------------------

    resp = []

    # The BGR2RGB conversion will be done in the preprocessing step of retinaface.
    # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #retinaface expects RGB but OpenCV read BGR

    """
    face = None
    img_region = [0, 0, img.shape[0], img.shape[1]] #Really?
    faces = RetinaFace.extract_faces(img_rgb, model = face_detector, align = align)
    if len(faces) > 0:
        face = faces[0][:, :, ::-1]
    return face, img_region
    """

    #--------------------------

    obj = RetinaFace.detect_faces(img, model = face_detector, threshold = 0.9)

    if type(obj) == dict:
        for key in obj:
            identity = obj[key]
            facial_area = identity["facial_area"]

            y = facial_area[1]
            h = facial_area[3] - y
            x = facial_area[0]
            w = facial_area[2] - x
            img_region = [x, y, w, h]

            #detected_face = img[int(y):int(y+h), int(x):int(x+w)] #opencv
            detected_face = img[facial_area[1]: facial_area[3], facial_area[0]: facial_area[2]]

            if align:
                landmarks = identity["landmarks"]
                left_eye = landmarks["left_eye"]
                right_eye = landmarks["right_eye"]
                nose = landmarks["nose"]
                #mouth_right = landmarks["mouth_right"]
                #mouth_left = landmarks["mouth_left"]

                detected_face = postprocess.alignment_procedure(detected_face, right_eye, left_eye, nose)

            resp.append((detected_face, img_region))

    return resp

In [ ]:
def anonymize_face_pixelate(image, blocks=3):
	# divide the input image into NxN blocks
	(h, w) = image.shape[:2]
	xSteps = np.linspace(0, w, blocks + 1, dtype="int")
	ySteps = np.linspace(0, h, blocks + 1, dtype="int")
	# loop over the blocks in both the x and y direction
	for i in range(1, len(ySteps)):
		for j in range(1, len(xSteps)):
			# compute the starting and ending (x, y)-coordinates
			# for the current block
			startX = xSteps[j - 1]
			startY = ySteps[i - 1]
			endX = xSteps[j]
			endY = ySteps[i]
			# extract the ROI using NumPy array slicing, compute the
			# mean of the ROI, and then draw a rectangle with the
			# mean RGB values over the ROI in the original image
			roi = image[startY:endY, startX:endX]
			(B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
			cv2.rectangle(image, (startX, startY), (endX, endY),
				(B, G, R), -1)
	# return the pixelated blurred image
	return image

In [ ]:
cv2.CAP_PROP_FPS

In [ ]:
cap = cv2.VideoCapture('/Users/andreibirladeanu/Documents/Data/play_videos/1073_play.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')# note the lower case
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
out = cv2.VideoWriter('/Users/andreibirladeanu/Documents/Data/1073_play_blurred.mp4', fourcc, 25.0, (int(width),int(height)))

In [ ]:
while (cap.isOpened()):
    ret, image = cap.read()
    if ret!=True:
        break
    faces = detect_face(face_detector, image, align=True)
    print(cap.get(cv2.CAP_PROP_POS_FRAMES))
    for k in range(len(faces)):
        start_x, start_y, end_x, end_y = faces[k][1][0], faces[k][1][1], faces[k][1][0]+faces[k][1][2], faces[k][1][1]+faces[k][1][3]
        cropped = image[start_y:end_y, start_x:end_x]
        blur = anonymize_face_pixelate(cropped)
        image[start_y:end_y, start_x:end_x] = blur
    out.write(image)

cap.release()
out.release()
cv2.destroyAllWindows()
    

In [ ]:
cap.release() 

In [ ]:
for k in range(len(faces)):
    print(faces)

In [ ]:
plt.imshow(image[199:199+47, 290:290+32])

In [ ]:
plt.imshow(cv2.GaussianBlur(image[199:199+47, 290:290+32], (1,1), 0))

In [ ]:
face = detect_face(face_detector, image, align = True)

In [ ]:
face